# **A Hybrid Clinical Decision Support System for Mycetoma Diagnosis using Multimodal Deep Learning**
*Testing*

## **Importing Libraries**

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import tensorflow as tf
from PIL import Image
import joblib
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **1. Load Model and Scaler**

In [2]:
print("Loading the trained model and data scaler...")
try:
    model = load_model('/content/drive/My Drive/Project 2/mycetoma_hybrid_model.h5')
    scaler = joblib.load('/content/drive/My Drive/Project 2/clinical_data_scaler.joblib')
    print("Model and scaler loaded successfully.")
except (IOError, FileNotFoundError) as e:
    print(f"Error: {e}")
    print("Please run the training.py script first to train and save the model.")
    exit()

Loading the trained model and data scaler...


Model and scaler loaded successfully.


### **2. Generate and Preprocess New Unseen Data**

In [9]:
if not os.path.exists('test_data'):
    os.makedirs('test_data')
def generate_synthetic_image(is_mycetoma, img_size=(224, 224), filename=''):
    """Generates a simple synthetic image for testing."""
    image = np.zeros((*img_size, 3), dtype=np.uint8)
    image += np.random.randint(0, 50, size=image.shape, dtype=np.uint8)
    if is_mycetoma:
        cx, cy = np.random.randint(50, 170), np.random.randint(50, 170)
        w, h = np.random.randint(30, 80), np.random.randint(30, 80)
        color = (200, 150, 150)
        image[cy-h//2:cy+h//2, cx-w//2:cx+w//2] = color
    else:
        cx, cy = np.random.randint(80, 150), np.random.randint(80, 150)
        w, h = np.random.randint(10, 30), np.random.randint(10, 30)
        color = (100, 100, 100)
        image[cy-h//2:cy+h//2, cx-w//2:cx+w//2] = color
    img = Image.fromarray(image, 'RGB')
    img.save(filename)
    return filename
def load_and_preprocess_image(path):
    """Loads and preprocesses an image for model prediction."""
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [128, 128]) # Changed from [224, 224] to [128, 128]
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    image = tf.expand_dims(image, axis=0)
    return image

### Example 1: Data suggesting Mycetoma

In [10]:
new_clinical_data_1 = pd.DataFrame({
    'age': [35],
    'duration_months': [30],
    'lesion_size_cm': [10],
    'pain_level': [3]
})
new_image_path_1 = generate_synthetic_image(is_mycetoma=True, filename='test_data/test_mycetoma_case.png')

/tmp/ipython-input-1414669616.py:17: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(image, 'RGB')


### Example 2: Data suggesting another condition

In [12]:
new_clinical_data_2 = pd.DataFrame({
    'age': [60],
    'duration_months': [5],
    'lesion_size_cm': [2],
    'pain_level': [8]
})
new_image_path_2 = generate_synthetic_image(is_mycetoma=False, filename='test_data/test_non_mycetoma_case.png')

/tmp/ipython-input-1414669616.py:17: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(image, 'RGB')


## **3. Make Predictions**

In [13]:
def predict_case(clinical_df, image_path, case_name):
    """Preprocesses data and makes a prediction for a single case."""
    print(f"\n--- {case_name} ---")
    print("Clinical Data:")
    print(clinical_df)
    print(f"Image Path: {image_path}")
    clinical_scaled = scaler.transform(clinical_df)
    clinical_tensor = tf.convert_to_tensor(clinical_scaled, dtype=tf.float32)
    image_tensor = load_and_preprocess_image(image_path)
    prediction_proba = model.predict([image_tensor, clinical_tensor])
    prediction_class = np.argmax(prediction_proba, axis=1)[0]
    confidence = prediction_proba[0][prediction_class]
    diagnosis = "Mycetoma" if prediction_class == 1 else "Non-Mycetoma"
    print("\nPrediction Result:")
    print(f"Diagnosis: {diagnosis}")
    print(f"Confidence: {confidence:.2%}")
predict_case(new_clinical_data_1, new_image_path_1, "Test Case 1 (Simulated Mycetoma)")
predict_case(new_clinical_data_2, new_image_path_2, "Test Case 2 (Simulated Non-Mycetoma)")


--- Test Case 1 (Simulated Mycetoma) ---
Clinical Data:
   age  duration_months  lesion_size_cm  pain_level
0   35               30              10           3
Image Path: test_data/test_mycetoma_case.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

Prediction Result:
Diagnosis: Mycetoma
Confidence: 79.94%

--- Test Case 2 (Simulated Non-Mycetoma) ---
Clinical Data:
   age  duration_months  lesion_size_cm  pain_level
0   60                5               2           8
Image Path: test_data/test_non_mycetoma_case.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step

Prediction Result:
Diagnosis: Non-Mycetoma
Confidence: 100.00%
